In [119]:
import pandas as pd
import numpy as np
    
import xml.etree.ElementTree as ET

from datetime import datetime 
pd.set_option('display.max_columns', None)
from functools import reduce


In [13]:
dfa_1 = pd.read_csv('../assets/data/2019 UT-ASD/2019-01-01--2019-03-31 arrivals.csv')  
dfa_2 = pd.read_csv('../assets/data/2019 UT-ASD/2019-04-01--2019-06-30_arrivals.csv')  
dfa_3 = pd.read_csv('../assets/data/2019 UT-ASD/2019-07-01--2019-07-31_arrivals.csv')  
dfa_4 = pd.read_csv('../assets/data/2019 UT-ASD/2019-08-01--2019-08-31 arrivals.csv')  
dfa_5 = pd.read_csv('../assets/data/2019 UT-ASD/2019-09-01--2019-10-31_arrivals.csv') 
dfa_6 = pd.read_csv('../assets/data/2019 UT-ASD/2019-11-01--2019-11-30_arrivals.csv')  
dfa_7 = pd.read_csv('../assets/data/2019 UT-ASD/2019-12-01--2019-12-31-arrivals.csv')  

In [14]:
dfa_all = [dfa_1, dfa_2, dfa_3, dfa_4, dfa_5, dfa_6, dfa_7]

In [15]:
dfa_not_parsed = pd.concat(dfa_all)
dfa_not_parsed

,date,xml_obj,uuid
0,2019-01-01 00:19:01.914659+01:00,"<?xml version=""1.0"" encoding=""UTF-8""?><ns1:Put...",755ef744-0d52-11e9-b8b1-06550c001849
1,2019-01-01 00:43:02.633669+01:00,"<?xml version=""1.0"" encoding=""UTF-8""?><ns1:Put...",d01b3d84-0d55-11e9-b8b1-06550c001849
2,2019-01-01 01:20:59.756816+01:00,"<?xml version=""1.0"" encoding=""UTF-8""?><ns1:Put...",1d60a9bc-0d5b-11e9-b8b1-06550c001849
3,2019-01-01 01:28:35.627872+01:00,"<?xml version=""1.0"" encoding=""UTF-8""?><ns1:Put...",2d19105a-0d5c-11e9-b8b1-06550c001849
4,2019-01-01 01:30:22.532675+01:00,"<?xml version=""1.0"" encoding=""UTF-8""?><ns1:Put...",6cd1737c-0d5c-11e9-b8b1-06550c001849
...,...,...,...
64,2019-12-01 07:37:53.864708+01:00,"<?xml version=""1.0"" encoding=""UTF-8""?><ns1:Put...",1a683598-1405-11ea-9132-06550c001849
65,2019-12-01 07:38:25.012352+01:00,"<?xml version=""1.0"" encoding=""UTF-8""?><ns1:Put...",2cf8f24c-1405-11ea-9132-06550c001849
66,2019-12-01 07:41:26.346982+01:00,"<?xml version=""1.0"" encoding=""UTF-8""?><ns1:Put...",990e6b92-1405-11ea-9132-06550c001849
67,2019-12-01 07:50:01.015618+01:00,"<?xml version=""1.0"" encoding=""UTF-8""?><ns1:Put...",cbd2a844-1406-11ea-9132-06550c001849


In [16]:
def extract_station_info(data, node, prefix):
  if node:
    data[prefix + 'Code'].append(node.find('./ns2:StationCode', ns).text)
    data[prefix + 'UIC'].append(int(node.find('./ns2:UICCode', ns).text))
    data[prefix + 'Type'].append(int(node.find('./ns2:Type', ns).text))
  else:
    data[prefix + 'Code'].append(np.NaN)
    data[prefix + 'UIC'].append(np.NaN)
    data[prefix + 'Type'].append(np.NaN)

def parse_timestamp(date_string):
  return datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S.%fZ')

def encode_list(nodes, sufix):
  UIC_codes = [node.find(sufix, ns).text for node in nodes]
  return ';'.join(UIC_codes)

In [17]:
station_amsterdam = b'<ns2:RitStation><ns2:StationCode>ASD</ns2:StationCode>'
station_utrecht = b'<ns2:RitStation><ns2:StationCode>UT</ns2:StationCode>'
source_amsterdam = b'<ns2:TreinHerkomst InfoStatus=""Gepland""><ns2:StationCode>ASD</ns2:StationCode'
source_utrecht = b'<ns2:TreinHerkomst InfoStatus=""Gepland""><ns2:StationCode>UT</ns2:StationCode>'
destination_amsterdam = b'<ns2:TreinEindBestemming InfoStatus=""Gepland""><ns2:StationCode>AMS</ns2:StationCode>'
destination_utrecht = b'<ns2:TreinEindBestemming InfoStatus=""Gepland""><ns2:StationCode>UT</ns2:StationCode>' 

In [18]:
# namespace of the xml object
ns={
    'ns1d': 'urn:ndov:cdm:trein:reisinformatie:messages:5',
    'ns1a': 'urn:ndov:cdm:trein:reisinformatie:messages:dynamischeaankomststaat:1',
    'ns2': 'urn:ndov:cdm:trein:reisinformatie:data:4',
}

# dictionary that will save the data while loading it in
def arrival_data_template():

    return {
        'ObservationTime': [],

        # Ride
        'RideId':[],
        'RideTime': [],

        # Departure station
        'DepartureStationCode': [],
        'DepartureStationUIC': [],
        'DepartureStationType': [],

        # Train
        'TrainId': [],
        'TrainType': [],
        'TrainOperator': [],

        # Actual destination
        'DestinationStationCode': [],
        'DestinationStationUIC': [],
        'DestinationStationType': [],

        # Arrival times
        'PlannedArrivalTime': [],
        'ActualArrivalTime': [],

        # Arrival platform
        'PlannedArrivalPlatform': [],
        'PlannedArrivalPlatformSuffix': [],
        'ActualArrivalPlatform': [],
        'ActualArrivalPlatformSuffix': [],

#         # Departure platforms
#         'PlannedDeparturePlatform': [],
#         'ActualDeparturePlatform': [],

#         # Stop stations
#         'PlannedStopStations': [],
#         'ActualStopStations': [],

#         # Matirial type
#         'MaterialType': [],
#         'MaterialDesignation': [],
#         'MaterialLength': [],

#         'ChangeType': [],

    }

def departure_data_template():
    return {
    'ObservationTime': [],

    # Ride
    'RideId':[],
    'RideTime': [],

    # Departure station
    'DepartureStationCode': [],
    'DepartureStationUIC': [],
    'DepartureStationType': [],

    # Train
    'TrainId': [],
    'TrainType': [],
    'TrainOperator': [],

    # Actual destination
    'DestinationStationCode': [],
    'DestinationStationUIC': [],
    'DestinationStationType': [],

    # Departure times
    'PlannedDepartureTime': [],
    'ActualDepartureTime': [],

    # Departure platform
    'PlannedDeparturePlatform': [],
    'PlannedDeparturePlatformSuffix': [],
    'ActualDeparturePlatform': [],
    'ActualDeparturePlatformSuffix': [],

    # Departure platforms
    'PlannedDeparturePlatform': [],
    'ActualDeparturePlatform': [],

    # Stop stations
    'PlannedStopStations': [],
    'ActualStopStations': [],

    # Matirial type
    'MaterialType': [],
    'MaterialDesignation': [],
    'MaterialLength': [],
    
    'HasChange': [],
    'ChangeType': [],

}

In [20]:
arrival_data = arrival_data_template()

# loop through every day

for root_text, observation_time in dfa_not_parsed[['xml_obj', 'date']].values:  
            
    arrival_data['ObservationTime'].append(observation_time)

    root = ET.fromstring(root_text)

    # Ride
    ride = root.find('./ns2:ReisInformatieProductDAS/ns2:DynamischeAankomstStaat', ns)

    arrival_data['RideId'].append(int(ride.find('./ns2:RitId', ns).text))
    arrival_data['RideTime'].append(root.find('./ns2:ReisInformatieProductDAS/ns2:RIPAdministratie/ns2:ReisInformatieTijdstip', ns).text)

    # Stations
    train = ride.find('./ns2:TreinAankomst', ns)

    destinationStation = ride.find('./ns2:RitStation', ns)
    departureStation = train.find('./ns2:TreinHerkomst[@InfoStatus="Gepland"]',ns)
    
    extract_station_info(arrival_data, departureStation, 'DepartureStation')
    extract_station_info(arrival_data, destinationStation, 'DestinationStation')

    # Arrival times
    arrival_data['ActualArrivalTime'].append(train.find('./ns2:AankomstTijd[@InfoStatus="Actueel"]', ns).text)
    arrival_data['PlannedArrivalTime'].append(train.find('./ns2:AankomstTijd[@InfoStatus="Gepland"]', ns).text)
    
    # Train
    arrival_data['TrainId'].append(train.find('./ns2:TreinNummer', ns).text)
    arrival_data['TrainType'].append(train.find('./ns2:TreinSoort', ns).text)
    arrival_data['TrainOperator'].append(train.find('./ns2:Vervoerder', ns).text)

    arrival_data['PlannedArrivalPlatform'].append(train.find('./ns2:TreinAankomstSpoor[@InfoStatus="Gepland"]/ns2:SpoorNummer', ns).text)
    suffix = train.find('./ns2:TreinAankomstSpoor[@InfoStatus="Gepland"]/ns2:SpoorFase', ns)
    arrival_data['PlannedArrivalPlatformSuffix'].append(None if suffix is None else suffix.text)

    arrival_data['ActualArrivalPlatform'].append(train.find('./ns2:TreinAankomstSpoor[@InfoStatus="Actueel"]/ns2:SpoorNummer', ns).text)
    suffix = train.find('./ns2:TreinAankomstSpoor[@InfoStatus="Actueel"]/ns2:SpoorFase', ns)
    arrival_data['ActualArrivalPlatformSuffix'].append(None if suffix is None else suffix.text)
    
# Convert the dictionary to a dataframe
dfa = pd.DataFrame(arrival_data)    
print(dfa.shape) # show the file size kinda
# # save the file, specify your path and name for the file
# df_flat.to_csv('./data/september.csv.zip', 
#                index=False, 
#                compression=dict(method='zip', archive_name='january.csv'))



(29885, 18)


In [21]:
dfa.head()

,ObservationTime,RideId,RideTime,DepartureStationCode,DepartureStationUIC,DepartureStationType,TrainId,TrainType,TrainOperator,DestinationStationCode,DestinationStationUIC,DestinationStationType,PlannedArrivalTime,ActualArrivalTime,PlannedArrivalPlatform,PlannedArrivalPlatformSuffix,ActualArrivalPlatform,ActualArrivalPlatformSuffix
0,2019-01-01 00:19:01.914659+01:00,1405,2019-01-01T00:29:00.000Z,UT,8400621,6,1405,Intercity,NS,ASD,8400058,6,2019-01-01T00:29:00.000Z,2019-01-01T00:29:00.000Z,2,a,2,a
1,2019-01-01 00:43:02.633669+01:00,1402,2019-01-01T00:53:00.000Z,ASD,8400058,6,1402,Intercity,NS,UT,8400621,6,2019-01-01T00:53:00.000Z,2019-01-01T00:53:00.000Z,15,None,15,None
2,2019-01-01 01:20:59.756816+01:00,1405,2019-01-01T00:29:00.000Z,UT,8400621,6,1405,Intercity,NS,ASD,8400058,6,2019-01-01T00:29:00.000Z,2019-01-01T00:30:49.000Z,2,a,2,a
3,2019-01-01 01:28:35.627872+01:00,1405,2019-01-01T00:29:00.000Z,UT,8400621,6,1405,Intercity,NS,ASD,8400058,6,2019-01-01T00:29:00.000Z,2019-01-01T00:29:31.000Z,2,a,2,a
4,2019-01-01 01:30:22.532675+01:00,1405,2019-01-01T00:29:00.000Z,UT,8400621,6,1405,Intercity,NS,ASD,8400058,6,2019-01-01T00:29:00.000Z,2019-01-01T00:29:31.000Z,2,a,2,a


In [23]:
dfa.shape

(29885, 18)

In [29]:
dfa.duplicated().sum()

0

In [31]:
if 'RideInstance' in dfa:
    del dfa['RideInstance']
dfa.insert(0, 'RideInstance', dfa.RideId.astype(str) + '#' + dfa.RideTime )

In [32]:
dfa.head()

,RideInstance,ObservationTime,RideId,RideTime,DepartureStationCode,DepartureStationUIC,DepartureStationType,TrainId,TrainType,TrainOperator,DestinationStationCode,DestinationStationUIC,DestinationStationType,PlannedArrivalTime,ActualArrivalTime,PlannedArrivalPlatform,PlannedArrivalPlatformSuffix,ActualArrivalPlatform,ActualArrivalPlatformSuffix
0,1405#2019-01-01T00:29:00.000Z,2019-01-01 00:19:01.914659+01:00,1405,2019-01-01T00:29:00.000Z,UT,8400621,6,1405,Intercity,NS,ASD,8400058,6,2019-01-01T00:29:00.000Z,2019-01-01T00:29:00.000Z,2,a,2,a
1,1402#2019-01-01T00:53:00.000Z,2019-01-01 00:43:02.633669+01:00,1402,2019-01-01T00:53:00.000Z,ASD,8400058,6,1402,Intercity,NS,UT,8400621,6,2019-01-01T00:53:00.000Z,2019-01-01T00:53:00.000Z,15,None,15,None
2,1405#2019-01-01T00:29:00.000Z,2019-01-01 01:20:59.756816+01:00,1405,2019-01-01T00:29:00.000Z,UT,8400621,6,1405,Intercity,NS,ASD,8400058,6,2019-01-01T00:29:00.000Z,2019-01-01T00:30:49.000Z,2,a,2,a
3,1405#2019-01-01T00:29:00.000Z,2019-01-01 01:28:35.627872+01:00,1405,2019-01-01T00:29:00.000Z,UT,8400621,6,1405,Intercity,NS,ASD,8400058,6,2019-01-01T00:29:00.000Z,2019-01-01T00:29:31.000Z,2,a,2,a
4,1405#2019-01-01T00:29:00.000Z,2019-01-01 01:30:22.532675+01:00,1405,2019-01-01T00:29:00.000Z,UT,8400621,6,1405,Intercity,NS,ASD,8400058,6,2019-01-01T00:29:00.000Z,2019-01-01T00:29:31.000Z,2,a,2,a


In [37]:
dfa.sort_values(['RideInstance', 'ActualArrivalTime'], ascending = True, inplace = True)
dfa

,RideInstance,ObservationTime,RideId,RideTime,DepartureStationCode,DepartureStationUIC,DepartureStationType,TrainId,TrainType,TrainOperator,DestinationStationCode,DestinationStationUIC,DestinationStationType,PlannedArrivalTime,ActualArrivalTime,PlannedArrivalPlatform,PlannedArrivalPlatformSuffix,ActualArrivalPlatform,ActualArrivalPlatformSuffix
15090,104#2019-06-22T19:58:00.000Z,2019-06-22 21:03:14.027447+02:00,104,2019-06-22T19:58:00.000Z,UT,8400621,6,300104,ICE International,NS,ASD,8400058,6,2019-06-22T19:58:00.000Z,2019-06-22T19:58:00.000Z,7,b,7,b
15092,104#2019-06-22T19:58:00.000Z,2019-06-22 21:16:20.243615+02:00,104,2019-06-22T19:58:00.000Z,UT,8400621,6,300104,ICE International,NS,ASD,8400058,6,2019-06-22T19:58:00.000Z,2019-06-22T19:58:00.000Z,7,b,7,b
129,105#2019-01-02T07:33:00.000Z,2019-01-02 07:23:08.704680+01:00,105,2019-01-02T07:33:00.000Z,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-02T07:33:00.000Z,2019-01-02T07:33:00.000Z,18,None,18,None
132,105#2019-01-02T07:33:00.000Z,2019-01-02 08:16:34.890657+01:00,105,2019-01-02T07:33:00.000Z,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-02T07:33:00.000Z,2019-01-02T07:33:12.000Z,18,None,18,None
133,105#2019-01-02T07:33:00.000Z,2019-01-02 08:32:18.975099+01:00,105,2019-01-02T07:33:00.000Z,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-02T07:33:00.000Z,2019-01-02T07:33:12.000Z,18,None,18,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15891,875#2019-06-30T18:31:00.000Z,2019-06-30 20:32:34.894893+02:00,875,2019-06-30T18:31:00.000Z,ASD,8400058,6,875,Intercity,NS,UT,8400621,6,2019-06-30T18:31:00.000Z,2019-06-30T18:33:15.000Z,18,None,18,None
15892,875#2019-06-30T18:31:00.000Z,2019-06-30 20:34:50.259980+02:00,875,2019-06-30T18:31:00.000Z,ASD,8400058,6,875,Intercity,NS,UT,8400621,6,2019-06-30T18:31:00.000Z,2019-06-30T18:33:15.000Z,18,None,18,None
24535,875#2019-09-29T18:31:00.000Z,2019-09-29 19:21:08.352184+02:00,875,2019-09-29T18:31:00.000Z,ASD,8400058,6,875,Intercity,NS,UT,8400621,6,2019-09-29T18:31:00.000Z,2019-09-29T18:31:00.000Z,18,None,18,None
24547,875#2019-09-29T18:31:00.000Z,2019-09-29 20:30:24.655274+02:00,875,2019-09-29T18:31:00.000Z,ASD,8400058,6,875,Intercity,NS,UT,8400621,6,2019-09-29T18:31:00.000Z,2019-09-29T18:31:00.000Z,18,None,18,None


In [39]:
dfa.drop_duplicates('RideInstance', keep='last', inplace = True)

In [42]:
dfa.duplicated(subset = ['RideInstance']).sum()

0

In [43]:
dfa.shape

(7571, 19)

In [44]:
dfa.to_csv('../assets/data/2019 UT-ASD/2019-parsed-data.csv')

In [71]:
 dfa['WeatherKey'] = dfa['RideTime'].dt.strftime('%Y-%m-%d-%H')

In [65]:
dfa['RideTime'].dtype
dfa['RideTime'] = pd.to_datetime(dfa['RideTime'])

In [72]:
dfa

,RideInstance,ObservationTime,RideId,RideTime,DepartureStationCode,DepartureStationUIC,DepartureStationType,TrainId,TrainType,TrainOperator,DestinationStationCode,DestinationStationUIC,DestinationStationType,PlannedArrivalTime,ActualArrivalTime,PlannedArrivalPlatform,PlannedArrivalPlatformSuffix,ActualArrivalPlatform,ActualArrivalPlatformSuffix,WeatherKey
15092,104#2019-06-22T19:58:00.000Z,2019-06-22 21:16:20.243615+02:00,104,2019-06-22 19:58:00+00:00,UT,8400621,6,300104,ICE International,NS,ASD,8400058,6,2019-06-22T19:58:00.000Z,2019-06-22T19:58:00.000Z,7,b,7,b,2019-06-22-19
131,105#2019-01-02T07:33:00.000Z,2019-01-02 08:14:53.961020+01:00,105,2019-01-02 07:33:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-02T07:33:00.000Z,2019-01-02T07:34:02.000Z,18,None,18,None,2019-01-02-07
206,105#2019-01-03T07:33:00.000Z,2019-01-03 08:49:03.881832+01:00,105,2019-01-03 07:33:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-03T07:33:00.000Z,2019-01-03T07:33:00.000Z,18,None,18,None,2019-01-03-07
276,105#2019-01-04T07:33:00.000Z,2019-01-04 08:37:38.178122+01:00,105,2019-01-04 07:33:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-04T07:33:00.000Z,2019-01-04T07:34:13.000Z,18,None,18,None,2019-01-04-07
354,105#2019-01-05T07:32:00.000Z,2019-01-05 08:40:55.071092+01:00,105,2019-01-05 07:32:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-05T07:32:00.000Z,2019-01-05T07:38:37.000Z,19,None,19,None,2019-01-05-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24538,871#2019-09-29T17:31:00.000Z,2019-09-29 19:33:54.061500+02:00,871,2019-09-29 17:31:00+00:00,ASD,8400058,6,871,Intercity,NS,UT,8400621,6,2019-09-29T17:31:00.000Z,2019-09-29T17:31:00.000Z,18,None,18,None,2019-09-29-17
15888,873#2019-06-30T18:01:00.000Z,2019-06-30 20:03:54.979467+02:00,873,2019-06-30 18:01:00+00:00,ASD,8400058,6,873,Intercity,NS,UT,8400621,6,2019-06-30T18:01:00.000Z,2019-06-30T18:01:00.000Z,18,None,18,None,2019-06-30-18
24543,873#2019-09-29T18:01:00.000Z,2019-09-29 20:04:15.945643+02:00,873,2019-09-29 18:01:00+00:00,ASD,8400058,6,873,Intercity,NS,UT,8400621,6,2019-09-29T18:01:00.000Z,2019-09-29T18:01:00.000Z,18,None,18,None,2019-09-29-18
15892,875#2019-06-30T18:31:00.000Z,2019-06-30 20:34:50.259980+02:00,875,2019-06-30 18:31:00+00:00,ASD,8400058,6,875,Intercity,NS,UT,8400621,6,2019-06-30T18:31:00.000Z,2019-06-30T18:33:15.000Z,18,None,18,None,2019-06-30-18


In [ ]:
dfa_weather_U = pd.read_csv('../assets/data/de_bilt_weather_2019.csv')  
dfa_weather_A = pd.read_csv('../assets/data/schiphol_weather_2019.csv') 

In [160]:
dfa_weather_A['WeatherKey'] = dfa_weather_A['Timestamp'].astype(str) + '-' + dfa_weather_A['Hour'].astype(str) + '-' + dfa_weather_A['StationCode'].astype(str)
dfa_weather_U['WeatherKey'] = dfa_weather_U['Timestamp'].astype(str) + '-' + dfa_weather_U['Hour'].astype(str)+ '-'+ dfa_weather_U['StationCode'].astype(str)

In [161]:
def add_uic_code(destination, weather):
    if destination == 'ASD':
        return f'{weather[0:13]}-240'
    if destination == 'UT':
        return f'{weather[0:13]}-260'

In [162]:
dfa['WeatherKey'] = dfa.apply(lambda row: add_uic_code(row['DestinationStationCode'], row['WeatherKey']), axis=1)

In [163]:
dfa

,RideInstance,ObservationTime,RideId,RideTime,DepartureStationCode,DepartureStationUIC,DepartureStationType,TrainId,TrainType,TrainOperator,DestinationStationCode,DestinationStationUIC,DestinationStationType,PlannedArrivalTime,ActualArrivalTime,PlannedArrivalPlatform,PlannedArrivalPlatformSuffix,ActualArrivalPlatform,ActualArrivalPlatformSuffix,WeatherKey
15092,104#2019-06-22T19:58:00.000Z,2019-06-22 21:16:20.243615+02:00,104,2019-06-22 19:58:00+00:00,UT,8400621,6,300104,ICE International,NS,ASD,8400058,6,2019-06-22T19:58:00.000Z,2019-06-22T19:58:00.000Z,7,b,7,b,2019-06-22-19-240
131,105#2019-01-02T07:33:00.000Z,2019-01-02 08:14:53.961020+01:00,105,2019-01-02 07:33:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-02T07:33:00.000Z,2019-01-02T07:34:02.000Z,18,None,18,None,2019-01-02-07-260
206,105#2019-01-03T07:33:00.000Z,2019-01-03 08:49:03.881832+01:00,105,2019-01-03 07:33:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-03T07:33:00.000Z,2019-01-03T07:33:00.000Z,18,None,18,None,2019-01-03-07-260
276,105#2019-01-04T07:33:00.000Z,2019-01-04 08:37:38.178122+01:00,105,2019-01-04 07:33:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-04T07:33:00.000Z,2019-01-04T07:34:13.000Z,18,None,18,None,2019-01-04-07-260
354,105#2019-01-05T07:32:00.000Z,2019-01-05 08:40:55.071092+01:00,105,2019-01-05 07:32:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-05T07:32:00.000Z,2019-01-05T07:38:37.000Z,19,None,19,None,2019-01-05-07-260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24538,871#2019-09-29T17:31:00.000Z,2019-09-29 19:33:54.061500+02:00,871,2019-09-29 17:31:00+00:00,ASD,8400058,6,871,Intercity,NS,UT,8400621,6,2019-09-29T17:31:00.000Z,2019-09-29T17:31:00.000Z,18,None,18,None,2019-09-29-17-260
15888,873#2019-06-30T18:01:00.000Z,2019-06-30 20:03:54.979467+02:00,873,2019-06-30 18:01:00+00:00,ASD,8400058,6,873,Intercity,NS,UT,8400621,6,2019-06-30T18:01:00.000Z,2019-06-30T18:01:00.000Z,18,None,18,None,2019-06-30-18-260
24543,873#2019-09-29T18:01:00.000Z,2019-09-29 20:04:15.945643+02:00,873,2019-09-29 18:01:00+00:00,ASD,8400058,6,873,Intercity,NS,UT,8400621,6,2019-09-29T18:01:00.000Z,2019-09-29T18:01:00.000Z,18,None,18,None,2019-09-29-18-260
15892,875#2019-06-30T18:31:00.000Z,2019-06-30 20:34:50.259980+02:00,875,2019-06-30 18:31:00+00:00,ASD,8400058,6,875,Intercity,NS,UT,8400621,6,2019-06-30T18:31:00.000Z,2019-06-30T18:33:15.000Z,18,None,18,None,2019-06-30-18-260


In [164]:
print(dfa_weather_A.head())
print(dfa_weather_U.head())

   StationCode   Timestamp  Hour  WindDir  WindHour  WindSpeed  MaxWindSpeed  \
0          240  2019-01-01     1      260       7.0        6.0          10.0   
1          240  2019-01-01     2      260       7.0        7.0          10.0   
2          240  2019-01-01     3      250       7.0        7.0          11.0   
3          240  2019-01-01     4      250       7.0        8.0          11.0   
4          240  2019-01-01     5      260       9.0        9.0          12.0   

   Temperature  MinTemp10M  DewPointTemp  SunshineDur  Radiation  PrecipDur  \
0          8.5         NaN           5.7          0.0          0        0.0   
1          8.6         NaN           5.1          0.0          0        0.0   
2          8.5         NaN           5.1          0.0          0        0.0   
3          8.2         NaN           5.4          0.0          0        0.0   
4          8.7         NaN           5.8          0.0          0        0.0   

   PrecipHour  AirPressure  Visibility  Clou

In [165]:
result_weather = pd.concat([dfa_weather_A, dfa_weather_U])

In [166]:
result_weather

,StationCode,Timestamp,Hour,WindDir,WindHour,WindSpeed,MaxWindSpeed,Temperature,MinTemp10M,DewPointTemp,SunshineDur,Radiation,PrecipDur,PrecipHour,AirPressure,Visibility,Cloudines,Humidity,WeatherCode,WeatherCodeIndicator,Fog,Rain,Snow,Thunder,IceFormation,WeatherKey
0,240,2019-01-01,1,260,7.0,6.0,10.0,8.5,NaN,5.7,0.0,0,0.0,0.0,1030.6,69.0,8.0,82,NaN,5,0.0,0.0,0.0,0.0,0.0,2019-01-01-1-240
1,240,2019-01-01,2,260,7.0,7.0,10.0,8.6,NaN,5.1,0.0,0,0.0,0.0,1030.1,75.0,8.0,78,NaN,5,0.0,0.0,0.0,0.0,0.0,2019-01-01-2-240
2,240,2019-01-01,3,250,7.0,7.0,11.0,8.5,NaN,5.1,0.0,0,0.0,0.0,1029.5,75.0,8.0,79,NaN,5,0.0,0.0,0.0,0.0,0.0,2019-01-01-3-240
3,240,2019-01-01,4,250,7.0,8.0,11.0,8.2,NaN,5.4,0.0,0,0.0,0.0,1029.0,70.0,8.0,82,NaN,5,0.0,0.0,0.0,0.0,0.0,2019-01-01-4-240
4,240,2019-01-01,5,260,9.0,9.0,12.0,8.7,NaN,5.8,0.0,0,0.0,-0.1,1028.3,70.0,8.0,81,22.0,7,0.0,1.0,0.0,0.0,0.0,2019-01-01-5-240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,260,2019-12-31,20,130,2.0,3.0,5.0,4.0,NaN,3.7,0.0,0,0.0,0.0,1035.9,12.0,8.0,98,20.0,7,1.0,0.0,0.0,0.0,0.0,2019-12-31-20-260
8756,260,2019-12-31,21,140,3.0,4.0,6.0,4.0,NaN,3.5,0.0,0,0.0,0.0,1035.6,18.0,8.0,96,10.0,7,0.0,0.0,0.0,0.0,0.0,2019-12-31-21-260
8757,260,2019-12-31,22,120,4.0,4.0,7.0,3.5,NaN,3.2,0.0,0,0.0,0.0,1035.6,15.0,8.0,97,10.0,7,0.0,0.0,0.0,0.0,0.0,2019-12-31-22-260
8758,260,2019-12-31,23,130,2.0,2.0,5.0,1.6,NaN,1.4,0.0,0,0.0,0.0,1035.2,7.0,1.0,98,34.0,7,1.0,0.0,0.0,0.0,0.0,2019-12-31-23-260


In [167]:
result = pd.merge(dfa,result_weather, on = 'WeatherKey', how='left')
result

,RideInstance,ObservationTime,RideId,RideTime,DepartureStationCode,DepartureStationUIC,DepartureStationType,TrainId,TrainType,TrainOperator,DestinationStationCode,DestinationStationUIC,DestinationStationType,PlannedArrivalTime,ActualArrivalTime,PlannedArrivalPlatform,PlannedArrivalPlatformSuffix,ActualArrivalPlatform,ActualArrivalPlatformSuffix,WeatherKey,StationCode,Timestamp,Hour,WindDir,WindHour,WindSpeed,MaxWindSpeed,Temperature,MinTemp10M,DewPointTemp,SunshineDur,Radiation,PrecipDur,PrecipHour,AirPressure,Visibility,Cloudines,Humidity,WeatherCode,WeatherCodeIndicator,Fog,Rain,Snow,Thunder,IceFormation
0,104#2019-06-22T19:58:00.000Z,2019-06-22 21:16:20.243615+02:00,104,2019-06-22 19:58:00+00:00,UT,8400621,6,300104,ICE International,NS,ASD,8400058,6,2019-06-22T19:58:00.000Z,2019-06-22T19:58:00.000Z,7,b,7,b,2019-06-22-19-240,240.0,2019-06-22,19.0,50.0,6.0,5.0,9.0,19.1,NaN,13.0,1.0,47.0,0.0,0.0,1020.2,80.0,0.0,67.0,NaN,5.0,0.0,0.0,0.0,0.0,0.0
1,105#2019-01-02T07:33:00.000Z,2019-01-02 08:14:53.961020+01:00,105,2019-01-02 07:33:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-02T07:33:00.000Z,2019-01-02T07:34:02.000Z,18,None,18,None,2019-01-02-07-260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,105#2019-01-03T07:33:00.000Z,2019-01-03 08:49:03.881832+01:00,105,2019-01-03 07:33:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-03T07:33:00.000Z,2019-01-03T07:33:00.000Z,18,None,18,None,2019-01-03-07-260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,105#2019-01-04T07:33:00.000Z,2019-01-04 08:37:38.178122+01:00,105,2019-01-04 07:33:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-04T07:33:00.000Z,2019-01-04T07:34:13.000Z,18,None,18,None,2019-01-04-07-260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,105#2019-01-05T07:32:00.000Z,2019-01-05 08:40:55.071092+01:00,105,2019-01-05 07:32:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-05T07:32:00.000Z,2019-01-05T07:38:37.000Z,19,None,19,None,2019-01-05-07-260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7566,871#2019-09-29T17:31:00.000Z,2019-09-29 19:33:54.061500+02:00,871,2019-09-29 17:31:00+00:00,ASD,8400058,6,871,Intercity,NS,UT,8400621,6,2019-09-29T17:31:00.000Z,2019-09-29T17:31:00.000Z,18,None,18,None,2019-09-29-17-260,260.0,2019-09-29,17.0,230.0,5.0,5.0,12.0,16.6,NaN,15.5,0.0,6.0,0.7,0.5,993.4,58.0,8.0,93.0,81.0,7.0,0.0,1.0,0.0,0.0,0.0
7567,873#2019-06-30T18:01:00.000Z,2019-06-30 20:03:54.979467+02:00,873,2019-06-30 18:01:00+00:00,ASD,8400058,6,873,Intercity,NS,UT,8400621,6,2019-06-30T18:01:00.000Z,2019-06-30T18:01:00.000Z,18,None,18,None,2019-06-30-18-260,260.0,2019-06-30,18.0,280.0,3.0,3.0,6.0,22.3,22.4,12.7,1.0,103.0,0.0,0.0,1016.3,82.0,0.0,54.0,NaN,5.0,0.0,0.0,0.0,0.0,0.0
7568,873#2019-09-29T18:01:00.000Z,2019-09-29 20:04:15.945643+02:00,873,2019-09-29 18:01:00+00:00,ASD,8400058,6,873,Intercity,NS,UT,8400621,6,2019-09-29T18:01:00.000Z,2019-09-29T18:01:00.000Z,18,None,18,None,2019-09-29-18-260,260.0,2019-09-29,18.0,230.0,5.0,5.0,11.0,16.6,14.9,15.1,0.0,1.0,0.4,0.1,993.0,65.0,8.0,91.0,23.0,7.0,0.0,1.0,0.0,0.0,0.0
7569,875#2019-06-30T18:31:00.000Z,2019-06-30 20:34:50.259980+02:00,875,2019-06-30 18:31:00+00:00,ASD,8400058,6,875,Intercity,NS,UT,8400621,6,2019-06-30T18:31:00.000Z,2019-06-30T18:33:15.000Z,18,None,18,None,2019-06-30-18-260,260.0,2019-06-30,18.0,280.0,3.0,3.0,6.0,22.3,22.4,12.7,1.0,103.0,0.0,0.0,1016.3,82.0,0.0,54.0,NaN,5.0,0.0,0.0,0.0,0.0,0.0


In [124]:
dfs = [dfa, dfa_weather_A, dfa_weather_U]
result = reduce(lambda left,right: pd.merge(left,right,on='WeatherKey', how = 'left'), dfs)
result

,RideInstance,ObservationTime,RideId,RideTime,DepartureStationCode,DepartureStationUIC,DepartureStationType,TrainId,TrainType,TrainOperator,DestinationStationCode,DestinationStationUIC,DestinationStationType,PlannedArrivalTime,ActualArrivalTime,PlannedArrivalPlatform,PlannedArrivalPlatformSuffix,ActualArrivalPlatform,ActualArrivalPlatformSuffix,WeatherKey,StationCode_x,Timestamp_x,Hour_x,WindDir_x,WindHour_x,WindSpeed_x,MaxWindSpeed_x,Temperature_x,MinTemp10M_x,DewPointTemp_x,SunshineDur_x,Radiation_x,PrecipDur_x,PrecipHour_x,AirPressure_x,Visibility_x,Cloudines_x,Humidity_x,WeatherCode_x,WeatherCodeIndicator_x,Fog_x,Rain_x,Snow_x,Thunder_x,IceFormation_x,StationCode_y,Timestamp_y,Hour_y,WindDir_y,WindHour_y,WindSpeed_y,MaxWindSpeed_y,Temperature_y,MinTemp10M_y,DewPointTemp_y,SunshineDur_y,Radiation_y,PrecipDur_y,PrecipHour_y,AirPressure_y,Visibility_y,Cloudines_y,Humidity_y,WeatherCode_y,WeatherCodeIndicator_y,Fog_y,Rain_y,Snow_y,Thunder_y,IceFormation_y
0,104#2019-06-22T19:58:00.000Z,2019-06-22 21:16:20.243615+02:00,104,2019-06-22 19:58:00+00:00,UT,8400621,6,300104,ICE International,NS,ASD,8400058,6,2019-06-22T19:58:00.000Z,2019-06-22T19:58:00.000Z,7,b,7,b,2019-06-22-19-260,260.0,2019-06-22,19.0,30.0,4.0,4.0,7.0,20.3,NaN,11.7,1.0,46.0,0.0,0.0,1019.7,83.0,0.0,57.0,NaN,5.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,105#2019-01-02T07:33:00.000Z,2019-01-02 08:14:53.961020+01:00,105,2019-01-02 07:33:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-02T07:33:00.000Z,2019-01-02T07:34:02.000Z,18,None,18,None,2019-01-02-07-240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,105#2019-01-03T07:33:00.000Z,2019-01-03 08:49:03.881832+01:00,105,2019-01-03 07:33:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-03T07:33:00.000Z,2019-01-03T07:33:00.000Z,18,None,18,None,2019-01-03-07-240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,105#2019-01-04T07:33:00.000Z,2019-01-04 08:37:38.178122+01:00,105,2019-01-04 07:33:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-04T07:33:00.000Z,2019-01-04T07:34:13.000Z,18,None,18,None,2019-01-04-07-240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,105#2019-01-05T07:32:00.000Z,2019-01-05 08:40:55.071092+01:00,105,2019-01-05 07:32:00+00:00,ASD,8400058,6,105,ICE International,NS,UT,8400621,6,2019-01-05T07:32:00.000Z,2019-01-05T07:38:37.000Z,19,None,19,None,2019-01-05-07-240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7566,871#2019-09-29T17:31:00.000Z,2019-09-29 19:33:54.061500+02:00,871,2019-09-29 17:31:00+00:00,ASD,8400058,6,871,Intercity,NS,UT,8400621,6,2019-09-29T17:31:00.000Z,2019-09-29T17:31:00.000Z,18,None,18,None,2019-09-29-17-240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240.0,2019-09-29,17.0,240.0,9.0,10.0,14.0,17.4,NaN,15.3,0.5,12.0,0.0,0.0,992.6,69.0,8.0,87.0,NaN,5.0,0.0,0.0,0.0,0.0,0.0
7567,873#2019-06-30T18:01:00.000Z,2019-06-30 20:03:54.979467+02:00,873,2019-06-30 18:01:00+00:00,ASD,8400058,6,873,Intercity,NS,UT,84006